In [2]:
# Câu1: đọc csv bằng pandas và insert vào databse, 2 bảng 1 bảng company có company_id, với news_id. Bảng còn lại là news_infor chứa tất cả các thông tin của new trừ company_id.
# Câu 2: query lấy 2 cột, 1 cột là company_id, 1 cột là tổng số news trong company đấy.
# Câu 3: xong query 1 cty lấy cột company_id, count_news, và lấy news mới nhất nhé(xác định theo cột date_publish) 
# Câu4: xong query 1 cty lấy cột company_id, count_news, và lấy news mới nhất nhé(xác định theo cột date_publish). chỉ lấy những công ty có tổng số news >=100.
# Câu5: lấy các news của công ty id = 100000 và tính diff_date, diff_date là date_published của 2 bản ghi liền kề nhau. 
# ví dụ : news1 ngày 15
# news2 ngày 17=> diffdate = 2
# Câu6: lấy company_id, news_id, tính diff_date. (giống c5 nhưng tính cho tất cả cty chỉ tính diffdate cho những new cùng công ty)
# Câu6: lấy company_id, news_id, tính date_published (date_published mới thứ2 )
# Câu 7: tạo 1 bảng news_audit (news_id, date_published, type) . bảng này sẽ được insert data khi thêm sửa xóa 1 bản ghi trong news nhiệm vụ => tạo 3 trigger lắng nghe bảng news và insert vào bảng news_audit mỗi khi thêm sửa xóa  bản ghi trong news phải tự động insert vào bảng news_audit.
# Câu 8: 1 view này có cột date_published,prev_date_publishedz(là date_published của bản ghi sau) , date_diff.
# Câu9: đánh index để cải thiện tốc độ truy vấn câu 8

In [3]:
import pyodbc
import pandas as pd
import warnings
from sqlalchemy import create_engine
import urllib

warnings.filterwarnings('ignore')

Connect to database

In [4]:
Server   = "KHUONGPD\SQLEXPRESS"
Database = "Company Data"
UID      = "KhuongPD\khuon"

connection_str_with_db = ("Driver={ODBC Driver 17 for SQL Server};"
                  "Server=" + Server + ";"
                  "Database=" + Database + ";"
                  "Trusted_Connection=yes;")

try: 
    connection = pyodbc.connect(connection_str_with_db)
    print("Connected successfull")

except pyodbc.Error as e:
    sqlstate = e.args[1]
    print ("Unable to connect: ", sqlstate)

Connected successfull


In [5]:
data = pd.read_csv("data.csv", encoding='latin1')
data = data.where(pd.notnull(data), None)
data

,author,date_published,image_url,language,negative,neutral,news_id,positive,publisher,source,title,url,company_id
0,Adamya Sharma,7/16/2020,https://cdn57.androidauthority.net/wp-content/...,None,0.558876,0.422459,crunchbase_0000205c-8c88-4c55-9d5d-45d10803b3d8,0.018665,Android Authority,crunchbase,Heres how all those high-profile Twitter acco...,https://andauth.co/nCL75b,159286.0
1,Zachary Lutz,2/12/2012,http://www.blogcdn.com/www.engadget.com/media/...,None,0.020323,0.943212,crunchbase_000020a6-b224-49e4-bf28-957c1db69833,0.036465,Engadget,crunchbase,"Refresh Roundup: week of February 6th, 2012",http://www.engadget.com/2012/02/12/refresh-rou...,106634.0
2,Dan Bloom,3/19/2018,https://bcsecure01-a.akamaihd.net/24/422139600...,None,0.369676,0.603352,crunchbase_00017162-44ae-4b26-9fb9-b5150d0eb8f1,0.026972,The Mirror,crunchbase,EU and UK Brexit deal explained: What Britain ...,https://www.mirror.co.uk/news/politics/eu-uk-b...,121801.0
3,Nick Statt,5/17/2016,https://cdn3.vox-cdn.com/thumbor/NNT7CxL3iWVQf...,None,0.002309,0.117710,crunchbase_00019d63-c2d0-0c91-8506-9d1cb50566e5,0.879981,The Verge,crunchbase,Nest products gain a ton of new smart home int...,http://www.theverge.com/2016/5/17/11692172/nes...,113058.0
4,Anthony Ha,12/1/2014,http://tctechcrunch2011.files.wordpress.com/20...,None,0.024935,0.891139,crunchbase_0001f853-c4cf-4a1a-e0b6-11b8cb5a9d48,0.083926,TechCrunch,crunchbase,AOL Acquires Video Syndication Startup Vidible,http://techcrunch.com/2014/12/01/aol-acquires-...,106634.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500586,None,6/21/2022,None,None,0.001979,0.319663,dealroom_6410214,0.678358,PredictLeads,dealroom,Akbank receives award 2022 FICO® Decisions Award,https://www.businesswire.com/news/home/2022062...,112302.0
500587,None,6/23/2022,None,None,0.012815,0.944748,dealroom_6410223,0.042437,PredictLeads,dealroom,Box partnered with Fuji Xerox on Jun 23rd 22'.,https://www.businesswire.com/news/home/2022062...,143992.0
500588,None,2/26/2020,None,None,0.001207,0.225538,dealroom_6410774,0.773255,PredictLeads,dealroom,Kaminario Inc. recognized as MongoDB Enterpris...,http://www.flashsanworks.com/48171/kaminario-c...,131053.0
500589,None,2/26/2020,None,None,0.004128,0.756560,dealroom_6410776,0.239313,PredictLeads,dealroom,Kaminario Inc. launches five new assurances as...,http://www.flashsanworks.com/47585/kaminario-u...,131053.0


Câu1: đọc csv bằng pandas và insert vào databse, 2 bảng 1 bảng company có company_id, với news_id. Bảng còn lại là news_infor chứa tất cả các thông tin của new trừ company_id.

In [6]:
company = data[['company_id','news_id']]
company.head()

,company_id,news_id
0,159286.0,crunchbase_0000205c-8c88-4c55-9d5d-45d10803b3d8
1,106634.0,crunchbase_000020a6-b224-49e4-bf28-957c1db69833
2,121801.0,crunchbase_00017162-44ae-4b26-9fb9-b5150d0eb8f1
3,113058.0,crunchbase_00019d63-c2d0-0c91-8506-9d1cb50566e5
4,106634.0,crunchbase_0001f853-c4cf-4a1a-e0b6-11b8cb5a9d48


In [7]:
news_infor = data.drop(columns=['company_id'])
news_infor.head()

,author,date_published,image_url,language,negative,neutral,news_id,positive,publisher,source,title,url
0,Adamya Sharma,7/16/2020,https://cdn57.androidauthority.net/wp-content/...,None,0.558876,0.422459,crunchbase_0000205c-8c88-4c55-9d5d-45d10803b3d8,0.018665,Android Authority,crunchbase,Heres how all those high-profile Twitter acco...,https://andauth.co/nCL75b
1,Zachary Lutz,2/12/2012,http://www.blogcdn.com/www.engadget.com/media/...,None,0.020323,0.943212,crunchbase_000020a6-b224-49e4-bf28-957c1db69833,0.036465,Engadget,crunchbase,"Refresh Roundup: week of February 6th, 2012",http://www.engadget.com/2012/02/12/refresh-rou...
2,Dan Bloom,3/19/2018,https://bcsecure01-a.akamaihd.net/24/422139600...,None,0.369676,0.603352,crunchbase_00017162-44ae-4b26-9fb9-b5150d0eb8f1,0.026972,The Mirror,crunchbase,EU and UK Brexit deal explained: What Britain ...,https://www.mirror.co.uk/news/politics/eu-uk-b...
3,Nick Statt,5/17/2016,https://cdn3.vox-cdn.com/thumbor/NNT7CxL3iWVQf...,None,0.002309,0.117710,crunchbase_00019d63-c2d0-0c91-8506-9d1cb50566e5,0.879981,The Verge,crunchbase,Nest products gain a ton of new smart home int...,http://www.theverge.com/2016/5/17/11692172/nes...
4,Anthony Ha,12/1/2014,http://tctechcrunch2011.files.wordpress.com/20...,None,0.024935,0.891139,crunchbase_0001f853-c4cf-4a1a-e0b6-11b8cb5a9d48,0.083926,TechCrunch,crunchbase,AOL Acquires Video Syndication Startup Vidible,http://techcrunch.com/2014/12/01/aol-acquires-...


In [8]:
params = urllib.parse.quote_plus(connection_str_with_db)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
dataframes = {'company': company, 'news_infor': news_infor}

try:
    for name, df in dataframes.items():
        table_name = name
        df.to_sql(table_name, engine, if_exists='replace', index=False)
    print("Các DataFrame đã được lưu vào cơ sở dữ liệu.")
except Exception as e:
    print("Error saving DataFrame: ", e)

Các DataFrame đã được lưu vào cơ sở dữ liệu.


----------------------------------------------------------------------------------------------------------------------------------------------------------

Câu 2: query lấy 2 cột, 1 cột là company_id, 1 cột là tổng số news trong company đấy

In [9]:
query2 = """
        Select company_id, count(company_id) as NumberOfNews
        from company as c join news_infor as n on c.news_id = n.news_id
        group by company_id
        order by count(company_id) desc"""
df2 = pd.read_sql(query2, connection)
df2

,company_id,NumberOfNews
0,159286.0,13279
1,132624.0,8725
2,111617.0,7096
3,155634.0,6483
4,134146.0,6448
...,...,...
24630,120262.0,1
24631,143990.0,1
24632,147728.0,1
24633,107893.0,1


-----------------------------------------------------------------------------------------------------------------------------------------------------------

Câu 3: xong query 1 cty lấy cột company_id, count_news, và lấy news mới nhất nhé(xác định theo cột date_publish) 

In [10]:
query3 = """WITH LatestNews AS (
    SELECT c.company_id,
           n.news_id,
           n.date_published,
           ROW_NUMBER() OVER (PARTITION BY company_id ORDER BY date_published DESC) AS rn 
    FROM company as c join news_infor as n on c.news_id = n.news_id
),
NewsCount AS (
    SELECT company_id,
           COUNT(*) AS count_news
    FROM company as c join news_infor as n on c.news_id = n.news_id
    GROUP BY company_id
)
SELECT DISTINCT c.company_id, nc.count_news, ln.news_id
FROM company c
LEFT JOIN NewsCount nc
    ON c.company_id = nc.company_id
LEFT JOIN LatestNews ln
    ON c.company_id = ln.company_id AND ln.rn = 1
    order by count_news desc;
"""
df3 = pd.read_sql(query3,connection)
df3

,company_id,count_news,news_id
0,159286.0,13279.0,crunchbase_bf77269f-bc51-4272-bc64-30f8d7710e9e
1,132624.0,8725.0,crunchbase_fa145b57-9bc9-419e-9fca-6516e0c6e60d
2,111617.0,7096.0,crunchbase_2289b19d-d51a-41e3-8178-8f306840d10f
3,155634.0,6483.0,crunchbase_c6e4ee1e-a987-4aa7-9018-27a5b9a69613
4,134146.0,6448.0,crunchbase_0de18aff-b1a3-424e-93ab-7c6e48585b29
...,...,...,...
24630,166335.0,1.0,dealroom_8664972
24631,166561.0,1.0,dealroom_4015184
24632,166594.0,1.0,dealroom_8902416
24633,166697.0,1.0,crunchbase_ce2d1831-aa13-4f6f-8793-754119284dba


Đích là 3 cột: company_id | count_news | news mới nhất
Trong đó news mới nhất được biểu diễn bởi news_id - Lý do news_id là unique key cho mỗi news
- Tạo 1 bảng con gồm company_id | count_news
- Tạo 1 bảng chứa company_id | news | Số gán cho các news 
    - Xác định news mới nhất:   
    Sử dụng PARTITION để chia thành các nhóm với key là company_id => thu được các nhóm tin ở các công ty khác nhau  
    Sử dụng ROW_NUMBER() ... OVER ... để đánh dấu số cho các tin trong từng nhóm được xác định từ PARTITION theo thứ tự giảm dần(mới nhất đánh số 1, mới nhì đánh số 2, ...)
- Join 2 bảng con lại theo company_id, lấy những hàng được gán số là 1 để thu được kết quả. Do bảng lúc này join với key là company_id nên dẫn đến trùng lặp nhiều hàng có nội dung giống nhau nên sử dụng SELECT DISTINCT để loại bỏ trùng lặp.
    

-----------------------------------------------------------------------------------------------------------------------------------------------------------

Câu4: xong query 1 cty lấy cột company_id, count_news, và lấy news mới nhất nhé(xác định theo cột date_publish). chỉ lấy những công ty có tổng số news >=100.

In [11]:
query4 = """WITH LatestNews AS (
    SELECT c.company_id,
           n.news_id,
           n.date_published,
           ROW_NUMBER() OVER (PARTITION BY company_id ORDER BY date_published DESC) AS rn
    FROM company as c join news_infor as n on c.news_id = n.news_id
),
NewsCount AS (
    SELECT company_id,
           COUNT(*) AS count_news
    FROM company as c join news_infor as n on c.news_id = n.news_id
    GROUP BY company_id
	HAVING COUNT(*) >= 100
)
SELECT DISTINCT c.company_id, nc.count_news, ln.news_id
FROM company c
LEFT JOIN NewsCount nc
    ON c.company_id = nc.company_id
LEFT JOIN LatestNews ln
    ON c.company_id = ln.company_id AND ln.rn = 1
WHERE nc.count_news IS NOT NULL
  AND ln.news_id IS NOT NULL
order by count_news desc;
"""
df4 = pd.read_sql(query4, connection)
df4

,company_id,count_news,news_id
0,159286.0,13279,crunchbase_bf77269f-bc51-4272-bc64-30f8d7710e9e
1,132624.0,8725,crunchbase_fa145b57-9bc9-419e-9fca-6516e0c6e60d
2,111617.0,7096,crunchbase_2289b19d-d51a-41e3-8178-8f306840d10f
3,155634.0,6483,crunchbase_c6e4ee1e-a987-4aa7-9018-27a5b9a69613
4,134146.0,6448,crunchbase_0de18aff-b1a3-424e-93ab-7c6e48585b29
...,...,...,...
1028,105958.0,100,crunchbase_5f7ad8e1-1c48-c074-96eb-af4377f0b4dd
1029,111409.0,100,crunchbase_9d3a21f8-546c-af0e-4c62-f7fef55334c7
1030,132145.0,100,crunchbase_2a86a988-7919-4456-872d-b9e863f07746
1031,138675.0,100,dealroom_6116133


Đích là 3 cột company_id | count_news | news mới nhất   
Trong đó new mới nhất xác định theo date_published và chỉ lấy ra những count_news >= 100

- Tạo 1 bảng con gồm company_id | count_news  
    - Chỉ lấy những count_news >= 100
- Tạo 1 bảng chứa company_id | news | Số gán cho các news 
    - Xác định news mới nhất:   
    Sử dụng PARTITION để chia thành các nhóm với key là company_id => thu được các nhóm tin ở các công ty khác nhau  
    Sử dụng ROW_NUMBER() ... OVER ... để đánh dấu số cho các tin trong từng nhóm được xác định từ PARTITION theo thứ tự giảm dần(mới nhất đánh số 1, mới nhì đánh số 2, ...)
- Join 2 bảng con lại theo company_id, lấy những hàng được gán số là 1 để thu được kết quả.Do bảng lúc này join với key là company_id nên dẫn đến trùng lặp nhiều hàng có nội dung giống nhau nên sử dụng SELECT DISTINCT để loại bỏ trùng lặp.

- Chỉ thêm so với câu trước ở điều kiện >= 100 trong bảng con xác định count_news
------------------------------------------------------------------------------------------------------------------------------------------------------------

Câu5: lấy các news của công ty id = 100000 và tính diff_date, diff_date là date_published của 2 bản ghi liền kề nhau.   
ví dụ : news1 ngày 15  
news2 ngày 17=> diffdate = 2

In [20]:
query5 = """
WITH CompanyNews AS (
    SELECT 
        c.company_id,
        n.news_id,
        n.date_published,
        LAG(n.date_published) OVER (PARTITION BY c.company_id ORDER BY n.date_published) AS previous_date_published
    FROM 
        company as c join news_infor as n on c.news_id = n.news_id
    WHERE 
        company_id = 100000
)
SELECT distinct
    company_id,
    news_id,
    date_published,
    DATEDIFF(DAY, TRY_CONVERT(DATE, date_published, 101),TRY_CONVERT(DATE, previous_date_published, 101)) AS diff_date
FROM 
    CompanyNews
ORDER BY 
    date_published;
"""
df5 = pd.read_sql(query5,connection)
df5

,company_id,news_id,date_published,diff_date
0,100000.0,crunchbase_b6452a03-fea5-4778-a60a-b3c226dc1f0e,2018-01-29,NaN
1,100000.0,crunchbase_e09c11b0-a3eb-44ef-9a80-d1b288387935,2018-01-29,0.0
2,100000.0,crunchbase_be1a41ac-747e-415f-b36f-a01e833a9739,2021-07-16,-1264.0
3,100000.0,crunchbase_be1a41ac-747e-415f-b36f-a01e833a9739,2021-07-16,0.0
4,100000.0,dealroom_4148252,2021-07-19,-3.0
5,100000.0,dealroom_5626529,2022-02-18,-214.0
6,100000.0,dealroom_5626529,2022-02-18,0.0
7,100000.0,crunchbase_9c61f878-7510-4eea-ae8d-d0f8417aa62c,2022-08-11,-174.0
8,100000.0,crunchbase_087d41bf-93e8-455a-a2e8-8cf683f5bd93,2022-12-13,-124.0
9,100000.0,dealroom_7733662,2023-01-24,-42.0


2 cột cần có : Company_id | Diff_date
- Tạo 1 bảng phụ chứa các nội dung: company_id | news_id | date_published | previous_date_published   
(mục đích chút nữa tính hiệu của 2 cột date_published | previous_date_published)
    - Sử dụng PARTITION để chia thành các nhóm với key là company_id => thu được các nhóm tin ở các công ty khác nhau, sắp xếp theo published_date
    - Từ bản ghi đã sắp xếp , Xác định previous_date_published thông qua hàm LAG(...) OVER (...) do hàm LAG trả về giá trị là bản ghi trước đó
    - Chỉ lấy những thông tin của công ty có id là 100000

- Select từ bảng phụ các nội dung cần thiết, trong đó có nhiệm vụ Diff_date
    - Xác định giá trị Diff_date thông qua hàm DATEDIFF(DAY, time1, time 2) = time1 - time2
        - DAY : định dạng dữ liệu
        - time1 : previous_date_published (đã được xác định từ bảng phụ)
        - time2: date_published
        - Tại từng giá trị thời gian, sử dụng TRY_CONVER(Kiểu dữ liệu, nội dung cần convert, định dạng) - định dạng 101 : day/month/year, 103: day-month-year. Nếu không convert được thì trả về NULL, dẫn đến tính toán Diff_date cũng trả về NULL
        
-----------------------------------------------------------------------------------------------------------------------------------------------------------

Câu6: lấy company_id, news_id, tính diff_date. (giống c5 nhưng tính cho tất cả cty chỉ tính diffdate cho những new cùng công ty)

In [18]:
query6 = """
WITH News AS (
    SELECT DISTINCT
        c.company_id,
        n.news_id,
        n.date_published,
        LAG(n.date_published) OVER (PARTITION BY c.company_id ORDER BY n.date_published) AS previous_date_published
    FROM 
        company AS c 
        JOIN news_infor AS n ON c.news_id = n.news_id
    WHERE c.company_id is NOT NULL
)
SELECT distinct
    company_id, 
    news_id,
    datediff(DAY,TRY_CONVERT(DATE, date_published, 101),TRY_CONVERT(DATE, previous_date_published, 101)) as Diffdate
FROM 
    News;
"""
df6 = pd.read_sql(query6,connection)
df6

,company_id,news_id,Diffdate
0,100000.0,crunchbase_087d41bf-93e8-455a-a2e8-8cf683f5bd93,-124.0
1,100000.0,crunchbase_9c61f878-7510-4eea-ae8d-d0f8417aa62c,-174.0
2,100000.0,crunchbase_b6452a03-fea5-4778-a60a-b3c226dc1f0e,NaN
3,100000.0,crunchbase_be1a41ac-747e-415f-b36f-a01e833a9739,-1264.0
4,100000.0,crunchbase_be1a41ac-747e-415f-b36f-a01e833a9739,0.0
...,...,...,...
591708,166801.0,dealroom_8169187,0.0
591709,166801.0,dealroom_8238464,-735.0
591710,166801.0,dealroom_8238464,0.0
591711,166801.0,dealroom_8512106,-92.0


- Tạo bảng phụ như câu nhưng chỉ cần loại bỏ WHERE ... để thao tác trên toàn bộ các công ty.
- Select từ bảng phụ với thao tác như câu 5
------------------------------------------------------------------------------------------------------------------------------------------------------------

Câu7: lấy company_id, news_id, tính date_published (date_published mới thứ2 )

In [14]:
query7 ="""
WITH RankedDates AS (
    SELECT 
        c.company_id,
		c.news_id,
        n.date_published,
        ROW_NUMBER() OVER (PARTITION BY c.company_id ORDER BY n.date_published DESC) AS rn
    FROM 
        news_infor n
    JOIN 
        company c ON n.news_id = c.news_id
)
SELECT 
    company_id, news_id,
    date_published AS second_latest_date_published
FROM 
    RankedDates
WHERE 
    rn = 2;
"""
df7 = pd.read_sql(query7, connection)
df7

,company_id,news_id,second_latest_date_published
0,NaN,dealroom_6902515,9/8/2022
1,100000.0,crunchbase_f225e5cf-b231-4c6f-88d9-713056538709,7/5/2023
2,100002.0,crunchbase_bd7fe00c-3b07-4d45-95be-541e497f29c0,9/28/2011
3,100003.0,crunchbase_b3c5c678-eaf8-ce9f-50ae-5c0ffa16ff5f,9/14/2016
4,100006.0,crunchbase_1399ce89-2888-40e4-bf5f-2755b5db7b64,2/28/2019
...,...,...,...
18971,166775.0,dealroom_8909787,3/5/2024
18972,166783.0,dealroom_8925191,3/7/2024
18973,166790.0,dealroom_8917078,3/5/2024
18974,166800.0,crunchbase_5380fd06-66f4-427b-8d42-c334580fc5d4,9/25/2019


Đích là 3: cột company_id | news_id | date_published_2nd
- Tạo 1 bảng phụ chứa company_id | news_id | date_published | cột đánh số cho date_published
    - Sử dụng PARTITION để phân nhóm theo company_id, sắp xếp theo date_published với thứ tự giảm dần (mới nhất đến cũ nhất)
    - Cột đánh số cho date_published được xác định bằng cách sử dụng hàm ROW_NUMBER ... OVER ... :mới nhất đánh số 1, mới nhì đánh số 2 ....

- Từ bảng phụ, chọn ra các cột, các giá trị thoả mãn date_published mới số 2 thông qua cột đánh số tại các giá trị là 2
----------------------------------------------------------------------------------------------------------------------------------------------------------

Câu 8: 1 view này có cột date_published,prev_date_publishedz(là date_published của bản ghi sau) , date_diff.

In [15]:
update_query = """
UPDATE news_infor
SET date_published = TRY_CONVERT(DATE, date_published, 101)
WHERE ISDATE(date_published) = 0;
"""

with connection.cursor() as cursor:
    cursor.execute(update_query)
    connection.commit()

query8 = """
WITH CompanyNews AS (
    SELECT 
        c.company_id,
        n.date_published,
        ROW_NUMBER() OVER (PARTITION BY c.company_id, n.date_published ORDER BY (SELECT NULL)) AS rn
    FROM 
        company AS c
    JOIN 
        news_infor AS n ON c.news_id = n.news_id
),
FilteredCompanyNews AS (
    SELECT 
        company_id,
        date_published
    FROM 
        CompanyNews
    WHERE 
        rn = 1
)
SELECT
    date_published,
    LEAD(date_published) OVER (PARTITION BY company_id ORDER BY date_published DESC) AS prev_date_published,
    DATEDIFF(
        DAY, 
        date_published, 
        LEAD(date_published) OVER (PARTITION BY company_id ORDER BY date_published DESC)
    ) AS Date_diff
FROM 
    FilteredCompanyNews;
"""

df8 = pd.read_sql(query8, connection)
df8

,date_published,prev_date_published,Date_diff
0,9/8/2022,9/8/2011,-4018.0
1,9/8/2011,9/7/2006,-1827.0
2,9/7/2006,9/30/2002,-1438.0
3,9/30/2002,9/29/2020,6574.0
4,9/29/2020,9/29/2013,-2557.0
...,...,...,...
366919,10/25/2017,10/23/2018,363.0
366920,10/23/2018,10/19/2017,-369.0
366921,10/19/2017,10/12/2023,2184.0
366922,10/12/2023,1/30/2019,-1716.0


3 cột đích: date_published | prev_date_published(là date_published của bản ghi sau) | date_diff
- Xác định bảng phụ 1 chứa company_id, các date_published : 
    - Sử dụng PARTITION BY ... để phân thành các nhóm, mỗi nhóm tuân thủ 2 điều kiện company_id và date_published. 1 công ty có thể đăng nhiều news trong 1 ngày => trùng lặp giá trị sau khi phân nhóm, tức là 1 nhóm sẽ gồm nhiều giá trị giống nhau. Vì vậy ta cần loại bỏ thông tin trùng lặp.
    - Sử dụng ROW_NUMBER ... OVER ... để thực hiện đánh số, ORDER BY (SELECT NULL) để thực hiện đánh số đơn thuần mà không cần sắp xếp.

- SELECT các cột và giá trị cần thiết ra thành bảng phụ 2 gọn gàng hơn, không có trùng lặp so với bảng số 1. Lấy giá trị được đánh số 1 do trong cùng nhóm(tuân thủ 2 điều kiện company_id và date_published), những giá trị còn lại là y hệt như giá trị được đánh số 1. Lúc này bảng gồm company_id | date_published
- Dựa vào date_published để xác định các bản ghi sau thông qua hàm LEAD ... OVER ... (ngược lại so với LAG: LAG trả về bản ghi trước đó)
- Có được date_published | prev_date_published	=> tiến hành tính toán date_diff bằng hàm DATEDIFF(DAY, time1, time2). Sau đó select để hiển thị các cột cần tìm.    
--
- Trước khi biến đổi, thực hiện chuyển kiểu dữ liệu về dạng thời gian qua hàm TRY_CONVERT
------------------------------------------------------------------------------------------------------------------------------------------------------------

Câu9: đánh index để cải thiện tốc độ truy vấn câu 8

In [16]:
check_column_query = """
SELECT DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'news_infor' AND COLUMN_NAME = 'date_published';
"""

df_check = pd.read_sql(check_column_query, connection)
print(df_check)

with connection.cursor() as cursor:
    if df_check.loc[0, 'DATA_TYPE'] != 'date' and df_check.loc[0, 'DATA_TYPE'] != 'datetime':
        cursor.execute("ALTER TABLE news_infor ALTER COLUMN date_published DATE;")
    connection.commit()

create_index_queries = [
    "CREATE INDEX idx_company_company_id ON company(company_id);",
    "CREATE INDEX idx_news_infor_date_published ON news_infor(date_published);"
]

with connection.cursor() as cursor:
    for query in create_index_queries:
        try:
            cursor.execute(query)
        except Exception as e:
            print(f"Error creating index: {e}")

update_query = """
UPDATE news_infor
SET date_published = TRY_CONVERT(DATE, date_published, 101)
WHERE TRY_CONVERT(DATE, date_published, 101) IS NOT NULL;
"""

with connection.cursor() as cursor:
    cursor.execute(update_query)
    connection.commit()

delete_invalid_query = """
DELETE FROM news_infor
WHERE TRY_CONVERT(DATE, date_published, 101) IS NULL;
"""

with connection.cursor() as cursor:
    cursor.execute(update_query)
    cursor.execute(delete_invalid_query)
    connection.commit()

query9 = """
WITH CompanyNews AS (
    SELECT 
        c.company_id,
        n.date_published,
        ROW_NUMBER() OVER (PARTITION BY c.company_id, n.date_published ORDER BY (SELECT NULL)) AS rn
    FROM 
        company AS c
    JOIN 
        news_infor AS n ON c.news_id = n.news_id
),
FilteredCompanyNews AS (
    SELECT 
        company_id,
        date_published
    FROM 
        CompanyNews
    WHERE 
        rn = 1
)
SELECT
    date_published,
    LEAD(date_published) OVER (PARTITION BY company_id ORDER BY date_published DESC) AS prev_date_published,
    DATEDIFF(
        DAY, 
        date_published, 
        LEAD(date_published) OVER (PARTITION BY company_id ORDER BY date_published DESC)
    ) AS Date_diff
FROM 
    FilteredCompanyNews;
"""

df9 = pd.read_sql(query9, connection)
df9

  DATA_TYPE
0   varchar


,date_published,prev_date_published,Date_diff
0,2024-03-07,2024-03-04,-3.0
1,2024-03-04,2024-02-29,-4.0
2,2024-02-29,2024-02-22,-7.0
3,2024-02-22,2024-02-15,-7.0
4,2024-02-15,2024-02-14,-1.0
...,...,...,...
366914,2017-10-06,2017-07-17,-81.0
366915,2017-07-17,2017-05-23,-55.0
366916,2017-05-23,2017-03-06,-78.0
366917,2017-03-06,2016-07-15,-234.0


- CREATE INDEX:  tạo các chỉ mục cho company_id và date_published
- Đoạn code check_column_query dùng để kiểm tra kiểu dữ liệu của date_published, ở đây kiểu dữ liệu đang là nvarchar => Trước khi mà thực hiện các thao tác, cần chuyển date_published về đúng kiểu dữ liệu DATE bằng TRY_CONVERT, cập nhật lại vào bảng.
- Thực hiện thao tác truy vấn.